# Compare results of two subgraphs


In [18]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px

load_dotenv()

pio.renderers.default = 'notebook'


In [19]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# mainnet
OLD_SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-goerli-perps'
SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/tburm/optimism-goerli-perps'
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# testnet
# SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-goerli-perps'
# RPC_ENDPOINT = f'https://optimism-kovan.infura.io/v3/{INFURA_KEY}'


In [20]:
# functions
convertDecimals = lambda x: Decimal(x) / Decimal(10**18)
convertBytes = lambda x: bytearray.fromhex(x[2:]).decode().replace('\x00', '')

def clean_df(df, decimal_cols=[], bytes_cols=[]):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    for col in bytes_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertBytes)
        else:
            print(f"{col} not in DataFrame")
    return df

async def run_query(query, params, endpoint=SUBGRAPH_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        df = pd.DataFrame(result)
        return df

async def run_recursive_query(query, params, accessor, endpoint=SUBGRAPH_ENDPOINT):
  transport = AIOHTTPTransport(url=endpoint)

  async with Client(
      transport=transport,
      fetch_schema_from_transport=True,
  ) as session:
    done_fetching = False
    all_results = []
    while not done_fetching:
      result = await session.execute(query, variable_values=params)
      if len(result[accessor]) > 0:
        all_results.extend(result[accessor])
        params['last_id'] = all_results[-1]['id']
      else:
        done_fetching = True
    
    df = pd.DataFrame(all_results)
    return df


In [21]:
position_query = gql("""
query positions(
    $last_id: ID!
  ) {
    futuresPositions(
      where: {
        id_gt: $last_id
      }
      # block: {number: 74087200}
      first: 1000
     ) {
      id
      account
      asset
      initialMargin
      size
      entryPrice
      avgEntryPrice
      lastPrice
      exitPrice
      pnl
      pnlWithFeesPaid
      openTimestamp
      closeTimestamp
      feesPaid
      netFunding
      isLiquidated
      isOpen
      margin
      netTransfers
      totalDeposits
      totalVolume
      lastTxHash
    }  
}
""")

trade_query = gql("""
query trades(
    $last_id: ID!
  ) {
    futuresTrades(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      timestamp
      asset
      marketKey
      orderType
      price
      margin
      positionId
      feesPaid
      pnl
      size
      positionSize
      positionClosed
    }  
}
""")

trade_compare_query = gql("""
query trades(
    $last_id: ID!
  ) {
    futuresTrades(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      timestamp
      asset
      marketKey
      orderType
      price
      margin
      positionId
      feesPaid
      keeperFeesPaid
      pnl
      size
      positionSize
      positionClosed
    }  
}
""")


### Run a query

In [22]:
trade_params = {
    'last_id': ''
}

trade_decimal_cols = [
    'price',
    'margin',
    'feesPaid',
    'pnl',
    'size',
    'positionSize',
]

trade_bytes_cols = [
    'marketKey',
    'asset',
]

df_trade = await run_recursive_query(trade_query, trade_params, 'futuresTrades')
df_trade = df_trade.replace({None: np.nan})
df_trade = clean_df(df_trade, decimal_cols=trade_decimal_cols,
                    bytes_cols=trade_bytes_cols)
df_trade['netPnl'] = df_trade['pnl'] - df_trade['feesPaid']


In [38]:
trade_compare_params = {
    'last_id': ''
}

trade_compare_decimal_cols = [
    'price',
    'margin',
    'feesPaid',
    'keeperFeesPaid',
    'pnl',
    'size',
    'positionSize',
]

trade_compare_bytes_cols = [
    'marketKey',
    'asset',
]

df_trade_compare = await run_recursive_query(trade_compare_query, trade_compare_params, 'futuresTrades')
df_trade_compare = df_trade_compare.replace({None: np.nan})
df_trade_compare = clean_df(df_trade_compare, decimal_cols=trade_compare_decimal_cols, bytes_cols=trade_compare_bytes_cols)
df_trade_compare['netPnl'] = df_trade_compare['pnl'] - df_trade_compare['feesPaid']


In [39]:
df_trade


,id,account,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,pnl,size,positionSize,positionClosed,netPnl
0,0x00b57a99bbf66c98b6a744edbb11c340b1cbec331f81...,0x7bdcd562c937facad6a955b9f2b2a7695b34ab9d,1678125434,sETH,sETHPERP,DelayedOffchain,1573.744176676525028046,206.004746485505368504,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x102,0.99969556156762679,0.037319881931439978,0.63523384320240064,0,True,-0.962375679636186812
1,0x00d3205eb2911b9dbce77366a3c92422f396f4f21c94...,0x76c568e643074553857ba1fb9b17f1ae5c7b0f34,1673500222,sETH,sETHPERP,DelayedOffchain,1403.690252208581322386,106.520902200981707443,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0xaa,1.274421444306777648,-0.609942728471391409,0.391,0,True,-1.884364172778169057
2,0x01eeebd24fd4e86f89c7892e45b0ecb5e2876dabb5e4...,0x76c568e643074553857ba1fb9b17f1ae5c7b0f34,1671423971,sETH,sETHPERP,DelayedOffchain,1184.204796650027170082,88.080778764988739163,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x79,1.044526100354041021,0,-0.0752,-0.0752,False,-1.044526100354041021
3,0x0239842c1420a53a2efcac2513b193af2dea0dda067e...,0x76758524bbc67e54e7f6c6967ab2f7be1d1c6ad8,1678034222,SOL,sSOLPERP,DelayedOffchain,21.1644217950772302,104827.318214942454841477,0x62068ebdcebcb0eb984abfea4c7f9a244050e0ca-0x7,1254.928819250701290489,0,-49411.8869,-49411.8869,False,-1254.928819250701290489
4,0x02d3974d461e4199901ecc6791434a714a876a5d9454...,0x3239a95a9262034ca28b9a03133775f716f119f8,1674846640,sETH,sETHPERP,DelayedOffchain,1605.556774458307569577,98.100770145795492927,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0xc1,1.050292937763580102,0.612083164480707555,-0.062648594635400841,0,True,-0.438209773282872547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,0xfca2c2f105cde5487a9bffa741147a875ebc7d3d95a2...,0x043e7c673f2bd9c62e69921395bfd1f97acc0a78,1675351372,sETH,sETHPERP,DelayedOffchain,1663.087154342313970399,2872.430621356671875791,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x87,13.167824626771141682,2965.62040619889395005,-8.6145,-2.0381,False,2952.452581572122808368
609,0xfe1aa04598d011e50b8ec531d71bf065f257b423e21e...,0x3239a95a9262034ca28b9a03133775f716f119f8,1675805200,XAG,sXAGPERP,DelayedOffchain,22.156500099995486845,99,0x78d1232449387571d652e2a893dc0feac6e92436-0x1,1.019999097461352126,0,4.513144533453683854,4.513144533453683854,False,-1.019999097461352126
610,0xfecf57f5e2870f34412ff6a1378bfbbbd77bdb0c1b0b...,0x76c568e643074553857ba1fb9b17f1ae5c7b0f34,1673223184,sETH,sETHPERP,DelayedOffchain,1290.483279980780431805,168.902907253339324663,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x9f,3.158849479079847584,-3.220365639827897277,1.6729,0,True,-6.379215118907744861
611,0xff62198dadffaaf867c7c0a3458b1bdef7c6797cc66e...,0x76c568e643074553857ba1fb9b17f1ae5c7b0f34,1676009734,sETH,sETHPERP,DelayedOffchain,1546.190046240109575566,126.842513205372398546,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0xdf,1.324158743194238972,-0.186601054241088696,-0.4193,0,True,-1.510759797435327668


In [40]:
df_trade_compare

,id,account,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,keeperFeesPaid,pnl,size,positionSize,positionClosed,netPnl
0,0x00b57a99bbf66c98b6a744edbb11c340b1cbec331f81...,0x7bdcd562c937facad6a955b9f2b2a7695b34ab9d,1678125434,sETH,sETHPERP,DelayedOffchain,1573.744176676525028046,206.004746485505368504,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x102,0.99969556156762679,0,0.037319881931439978,0.63523384320240064,0,True,-0.962375679636186812
1,0x00d3205eb2911b9dbce77366a3c92422f396f4f21c94...,0x76c568e643074553857ba1fb9b17f1ae5c7b0f34,1673500222,sETH,sETHPERP,DelayedOffchain,1403.690252208581322386,106.520902200981707443,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0xaa,1.274421444306777648,1,-0.609942728471391409,0.391,0,True,-1.884364172778169057
2,0x01eeebd24fd4e86f89c7892e45b0ecb5e2876dabb5e4...,0x76c568e643074553857ba1fb9b17f1ae5c7b0f34,1671423971,sETH,sETHPERP,DelayedOffchain,1184.204796650027170082,88.080778764988739163,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x79,1.044526100354041021,1,0,-0.0752,-0.0752,False,-1.044526100354041021
3,0x0239842c1420a53a2efcac2513b193af2dea0dda067e...,0x76758524bbc67e54e7f6c6967ab2f7be1d1c6ad8,1678034222,SOL,sSOLPERP,DelayedOffchain,21.1644217950772302,104827.318214942454841477,0x62068ebdcebcb0eb984abfea4c7f9a244050e0ca-0x7,1254.928819250701290489,0,0,-49411.8869,-49411.8869,False,-1254.928819250701290489
4,0x02d3974d461e4199901ecc6791434a714a876a5d9454...,0x3239a95a9262034ca28b9a03133775f716f119f8,1674846640,sETH,sETHPERP,DelayedOffchain,1605.556774458307569577,98.100770145795492927,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0xc1,1.050292937763580102,1,0.612083164480707555,-0.062648594635400841,0,True,-0.438209773282872547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,0xfca2c2f105cde5487a9bffa741147a875ebc7d3d95a2...,0x043e7c673f2bd9c62e69921395bfd1f97acc0a78,1675351372,sETH,sETHPERP,DelayedOffchain,1663.087154342313970399,2872.430621356671875791,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x87,13.167824626771141682,1,2965.62040619889395005,-8.6145,-2.0381,False,2952.452581572122808368
609,0xfe1aa04598d011e50b8ec531d71bf065f257b423e21e...,0x3239a95a9262034ca28b9a03133775f716f119f8,1675805200,XAG,sXAGPERP,DelayedOffchain,22.156500099995486845,99,0x78d1232449387571d652e2a893dc0feac6e92436-0x1,1.019999097461352126,1,0,4.513144533453683854,4.513144533453683854,False,-1.019999097461352126
610,0xfecf57f5e2870f34412ff6a1378bfbbbd77bdb0c1b0b...,0x76c568e643074553857ba1fb9b17f1ae5c7b0f34,1673223184,sETH,sETHPERP,DelayedOffchain,1290.483279980780431805,168.902907253339324663,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x9f,3.158849479079847584,1,-3.220365639827897277,1.6729,0,True,-6.379215118907744861
611,0xff62198dadffaaf867c7c0a3458b1bdef7c6797cc66e...,0x76c568e643074553857ba1fb9b17f1ae5c7b0f34,1676009734,sETH,sETHPERP,DelayedOffchain,1546.190046240109575566,126.842513205372398546,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0xdf,1.324158743194238972,1,-0.186601054241088696,-0.4193,0,True,-1.510759797435327668


In [41]:
df_trade.equals(df_trade_compare.drop('keeperFeesPaid', axis=1))

True

## Check the new stuff

In [42]:
df_trade_compare

,id,account,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,keeperFeesPaid,pnl,size,positionSize,positionClosed,netPnl
0,0x00b57a99bbf66c98b6a744edbb11c340b1cbec331f81...,0x7bdcd562c937facad6a955b9f2b2a7695b34ab9d,1678125434,sETH,sETHPERP,DelayedOffchain,1573.744176676525028046,206.004746485505368504,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x102,0.99969556156762679,0,0.037319881931439978,0.63523384320240064,0,True,-0.962375679636186812
1,0x00d3205eb2911b9dbce77366a3c92422f396f4f21c94...,0x76c568e643074553857ba1fb9b17f1ae5c7b0f34,1673500222,sETH,sETHPERP,DelayedOffchain,1403.690252208581322386,106.520902200981707443,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0xaa,1.274421444306777648,1,-0.609942728471391409,0.391,0,True,-1.884364172778169057
2,0x01eeebd24fd4e86f89c7892e45b0ecb5e2876dabb5e4...,0x76c568e643074553857ba1fb9b17f1ae5c7b0f34,1671423971,sETH,sETHPERP,DelayedOffchain,1184.204796650027170082,88.080778764988739163,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x79,1.044526100354041021,1,0,-0.0752,-0.0752,False,-1.044526100354041021
3,0x0239842c1420a53a2efcac2513b193af2dea0dda067e...,0x76758524bbc67e54e7f6c6967ab2f7be1d1c6ad8,1678034222,SOL,sSOLPERP,DelayedOffchain,21.1644217950772302,104827.318214942454841477,0x62068ebdcebcb0eb984abfea4c7f9a244050e0ca-0x7,1254.928819250701290489,0,0,-49411.8869,-49411.8869,False,-1254.928819250701290489
4,0x02d3974d461e4199901ecc6791434a714a876a5d9454...,0x3239a95a9262034ca28b9a03133775f716f119f8,1674846640,sETH,sETHPERP,DelayedOffchain,1605.556774458307569577,98.100770145795492927,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0xc1,1.050292937763580102,1,0.612083164480707555,-0.062648594635400841,0,True,-0.438209773282872547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,0xfca2c2f105cde5487a9bffa741147a875ebc7d3d95a2...,0x043e7c673f2bd9c62e69921395bfd1f97acc0a78,1675351372,sETH,sETHPERP,DelayedOffchain,1663.087154342313970399,2872.430621356671875791,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x87,13.167824626771141682,1,2965.62040619889395005,-8.6145,-2.0381,False,2952.452581572122808368
609,0xfe1aa04598d011e50b8ec531d71bf065f257b423e21e...,0x3239a95a9262034ca28b9a03133775f716f119f8,1675805200,XAG,sXAGPERP,DelayedOffchain,22.156500099995486845,99,0x78d1232449387571d652e2a893dc0feac6e92436-0x1,1.019999097461352126,1,0,4.513144533453683854,4.513144533453683854,False,-1.019999097461352126
610,0xfecf57f5e2870f34412ff6a1378bfbbbd77bdb0c1b0b...,0x76c568e643074553857ba1fb9b17f1ae5c7b0f34,1673223184,sETH,sETHPERP,DelayedOffchain,1290.483279980780431805,168.902907253339324663,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0x9f,3.158849479079847584,1,-3.220365639827897277,1.6729,0,True,-6.379215118907744861
611,0xff62198dadffaaf867c7c0a3458b1bdef7c6797cc66e...,0x76c568e643074553857ba1fb9b17f1ae5c7b0f34,1676009734,sETH,sETHPERP,DelayedOffchain,1546.190046240109575566,126.842513205372398546,0x111babcdd66b1b60a20152a2d3d06d36f8b5703c-0xdf,1.324158743194238972,1,-0.186601054241088696,-0.4193,0,True,-1.510759797435327668
